In [ ]:
#@title
# Upload: 
# data_iris.csv

# 6.3. Probabilistic Neural Networks (PNN)

In [ ]:
#@title 6.3.1. Import some necessary packages
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import MinMaxScaler


In [ ]:
#@title 6.3.2. PNN function

def fun_pnn(datain_tr, dataou_tr, datain_te, dataou_te, sigma_resolution = 0.01):
  def fun_norm(datain_tr,datain_te):
    num_tr = datain_tr.shape[0]
    num_te = datain_te.shape[0]

    rng_tr = np.array(range(0,num_tr,1))
    rng_te = np.array(range(0,num_te,1))

    ind_tr = rng_tr.repeat(num_te, axis = 0).reshape((num_tr, num_te)).transpose().reshape(num_tr * num_te,1)
    ind_te = rng_te.repeat(num_tr, axis = 0).transpose().reshape(num_tr * num_te,1)

    norm = np.expand_dims(np.sqrt(np.sum((datain_tr[ind_tr[:,0],:] - datain_te[ind_te[:,0],:])**2, axis = 1)), axis = 1)

    return norm, ind_tr, ind_te

  def fun_gaussian_pnn(datain_tr,datain_te,sigma, mode_val):
    I  = datain_tr.shape[1]
    norm, ind_tr, ind_te = fun_norm(datain_tr,datain_te)
    del ind_tr, ind_te

    fi = ( 1 / ( ((2*np.pi)**(I/2))*(sigma**I) ) ) *np.exp( -(norm)/(2*(sigma**2)) )
    
    if mode_val == 'tr':
      bad_val = ( 1 / ( ((2*np.pi)**(I/2))*(sigma**I) ) )
      fi[np.where(fi == bad_val)] = 0

    return fi

  def fun_pnn_train(datain_tr, dataou_tr, sigma_resolution):

    num_data  = datain_tr.shape[0]
    ind       = np.random.permutation(num_data)

    # validaiton data
    datain_te = datain_tr
    dataou_te = dataou_tr

    num_tr    = datain_tr.shape[0]
    num_te    = datain_te.shape[0]
    num_sigma = int(1/sigma_resolution)

    sigma         = np.array(range(1,num_sigma + 1,1))/num_sigma
    dataou        = np.concatenate((dataou_tr, dataou_te), axis = 0)
    dataou_unique = np.unique(dataou)
    num_classes   = dataou_unique.shape[0]

    fi      = fun_gaussian_pnn(datain_tr,datain_te,sigma, mode_val = 'tr')

    del datain_tr, datain_te

    p_class = np.empty((num_te, sigma.shape[0], num_classes))
    for i0 in range(num_classes):
      ind_ou          = np.where(dataou_tr == i0)[0]
      ind_class       = np.where(dataou_tr.repeat(num_te, axis = 1).transpose().reshape(num_tr * num_te,1)[:,0] == i0)[0]
      p_class[:,:,i0] = np.mean(fi[ind_class,:].reshape(num_te, ind_ou.shape[0], sigma.shape[0]), axis = 1)

    del fi

    class_val     = p_class.argmax(axis = 2)
    correct       = class_val == dataou_te.repeat(sigma.shape[0],axis = 1)
    
    del dataou_te, dataou_tr

    accuracy      = correct.sum(axis = 0)/num_te*100
    accuracy_tr  = accuracy.max() 
    sigma_optimum = np.where(accuracy == accuracy_tr)[0]/num_sigma

    return accuracy_tr, sigma_optimum

  def fun_pnn_test(datain_tr, dataou_tr, datain_te, dataou_te, sigma_optimum):
    num_tr    = datain_tr.shape[0]
    num_te    = datain_te.shape[0]

    sigma         = sigma_optimum
    dataou        = np.concatenate((dataou_tr, dataou_te), axis = 0)
    dataou_unique = np.unique(dataou)
    num_classes   = dataou_unique.shape[0]

    fi            = fun_gaussian_pnn(datain_tr,datain_te,sigma, mode_val = 'te')

    del datain_tr, datain_te

    p_class = np.empty((num_te, sigma.shape[0], num_classes))
    for i0 in range(num_classes):
      ind_ou          = np.where(dataou_tr == i0)[0]
      ind_class       = np.where(dataou_tr.repeat(num_te, axis = 1).transpose().reshape(num_tr * num_te,1)[:,0] == i0)[0]
      p_class[:,:,i0] = np.mean(fi[ind_class,:].reshape(num_te, ind_ou.shape[0], sigma.shape[0]), axis = 1)

    class_val     = p_class.argmax(axis = 2)
    correct       = class_val == dataou_te.repeat(sigma.shape[0],axis = 1)

    del dataou_te, dataou_tr

    accuracy      = correct.sum(axis = 0)/num_te*100
    return accuracy

  accuracy_tr, sigma_optimum = fun_pnn_train(datain_tr, dataou_tr, sigma_resolution)
  accuracy_te                = fun_pnn_test(datain_tr, dataou_tr, datain_te, dataou_te, sigma_optimum)

  return accuracy_te, accuracy_tr, sigma_optimum 

In [ ]:
#@title 6.3.3. Classification example: data preprocessing
data = pd.read_csv('/content/data_iris.csv')

datain = data.iloc[:,0:4]

dataou = data.iloc[:,-1:]
dataou[dataou == 'iris-setosa'] = 0
dataou[dataou == 'iris-virginica'] = 1
dataou[dataou == 'iris-versicolor'] = 2

datain = datain.values 
dataou = dataou.values

# object to float data
mat = np.empty(dataou.shape,dtype=float)
for i0 in range(dataou.shape[0]): mat[i0,0] = dataou[i0]
dataou = mat

# no need to convert int ocategorical/binary outputs 

datain_tr, datain_te, dataou_tr, dataou_te = train_test_split(datain, dataou, test_size = 0.1, random_state = 11)

scalerin = MinMaxScaler(feature_range=(0,1))
scalerin.fit(datain_tr)

datain_tr_calibrated = scalerin.transform(datain_tr)
datain_te_calibrated = scalerin.transform(datain_te)


In [ ]:
#@title 6.3.4. Classification example: training and testing

sigma_resolution = 0.1

accuracy_te, accuracy_tr, sigma_optimum = fun_pnn(datain_tr, dataou_tr, datain_te, dataou_te, sigma_resolution)


print("##########################################################")
print("##########################################################")
print(" ")
print("PNN Training Accuracy {}%".format(accuracy_tr))
print("PNN Testing Accuracy  {}%".format((accuracy_te)))
print(" ")
print("##########################################################")
print("##########################################################")


# 6.4.	Enhanced Probabilistic Neural Networks (EPNN)

In [ ]:
#@title 6.4.1. EPNN function

def fun_epnn(datain_tr, dataou_tr, datain_te, dataou_te, sigma_resolution = 0.1, radius_resolution = 0.1):
  def fun_norm(datain_tr,datain_te):
    num_tr = datain_tr.shape[0]
    num_te = datain_te.shape[0]

    rng_tr = np.array(range(0,num_tr,1))
    rng_te = np.array(range(0,num_te,1))

    ind_tr = rng_tr.repeat(num_te, axis = 0).reshape((num_tr, num_te)).transpose().reshape(num_tr * num_te,1)
    ind_te = rng_te.repeat(num_tr, axis = 0).transpose().reshape(num_tr * num_te,1)

    norm = np.expand_dims(np.sqrt(np.sum((datain_tr[ind_tr[:,0],:] - datain_te[ind_te[:,0],:])**2, axis = 1)), axis = 1)

    return norm, ind_tr, ind_te
    
  def fun_decision_hypersphere(datain_tr,dataou_tr, radius_resolution, sigma_resolution):
    num_radius           = int(1/radius_resolution)+1
    num_sigma            = int(1/sigma_resolution)
    radius               = np.expand_dims(np.array(range(0,num_radius    ,1))/num_radius, axis = 1)
    sigma                = np.expand_dims(np.array(range(1,num_sigma  + 1,1))/num_sigma , axis = 1)

    #print(radius[0])

    #print(radius.shape)
    #print(num_radius)

    num_tr               = datain_tr.shape[0]

    norm, ind_tr, ind_te = fun_norm(datain_tr,datain_tr)

    dataou_tr1           = dataou_tr[ind_tr[:,0],:]
    del ind_tr
    dataou_tr2           = dataou_tr[ind_te[:,0],:]                   
    del ind_te

    norm                 = np.expand_dims(norm.reshape(num_tr,num_tr)      , axis = 2)
    dataou_tr1           = np.expand_dims(dataou_tr1.reshape(num_tr,num_tr), axis = 2)
    dataou_tr2           = np.expand_dims(dataou_tr2.reshape(num_tr,num_tr), axis = 2)  

    norm                 = np.repeat(norm,       num_radius, axis = 2) 
    dataou_tr1           = np.repeat(dataou_tr1, num_radius, axis = 2) 
    dataou_tr2           = np.repeat(dataou_tr2, num_radius, axis = 2) 

    radius = np.expand_dims(radius, axis = 2)
    radius = radius.reshape((1,1,num_radius))
    radius = radius.repeat(num_tr, axis = 0)
    radius = radius.repeat(num_tr, axis = 1)

    sigma = np.expand_dims(sigma, axis = 2)
    sigma = sigma.reshape((1,1,num_sigma))
    sigma = sigma.repeat(num_tr, axis = 0)
    sigma = sigma.repeat(num_radius, axis = 1)

    denom = np.expand_dims(np.sum(norm <= radius, axis = 1), axis = 2)
    denom = denom.repeat(num_sigma, axis = 2)

    #print(dataou_tr1.shape)
    #print(dataou_tr2.shape)

    nomin_case1 = norm <= radius
    nomin_case2 = dataou_tr1 == dataou_tr2
    nomin_case3 = nomin_case1 * nomin_case2

    #print(nomin_case1.shape)
    #print(nomin_case2.shape)
    #print(nomin_case3.shape)

    del nomin_case1, nomin_case2

    nomin = np.expand_dims(np.sum(nomin_case3, axis = 1), axis = 2)
    #print(nomin.shape)
    nomin = nomin.repeat(num_sigma, axis = 2)

    del norm, dataou_tr1, dataou_tr2

    alpha = nomin / denom

    #print(nomin[:,0,0])
    #print(denom[:,0,0])

    #print(alpha[:,0,0])

    sigma = sigma * alpha

    return sigma

  def fun_gaussian_epnn(datain_tr, datain_te, sigma, mode_val):
    norm, ind_tr, ind_te = fun_norm(datain_tr,datain_te)
    del ind_te
    norm  = np.expand_dims(norm,axis=2)
    norm  = norm.repeat(sigma.shape[1], axis = 1)
    norm  = norm.repeat(sigma.shape[2], axis = 2)
    I     = datain_tr.shape[1]
    sigma = sigma[ind_tr[:,0],:,:]
    fi    = ( 1 / ( ((2*np.pi)**(I/2))*(sigma**I) ) ) * np.exp( -(norm)/(2*(sigma**2)) )
    if mode_val == 'tr':
      bad_val = ( 1 / ( ((2*np.pi)**(I/2))*(sigma**I) ) )
      del sigma, norm
      fi[np.where(fi == bad_val)] = 0
    else:
      del sigma, norm

    return fi

  def fun_epnn_train(datain_tr, dataou_tr, sigma_resolution):

      # validaiton data
    datain_te = datain_tr
    dataou_te = dataou_tr

    num_tr     = datain_tr.shape[0]
    num_te     = datain_te.shape[0]
    num_sigma  = int(1/sigma_resolution)
    num_radius = int(1/radius_resolution)+1

    sigma         = fun_decision_hypersphere(datain_tr,dataou_tr, radius_resolution, sigma_resolution)
    dataou        = np.concatenate((dataou_tr, dataou_te), axis = 0)
    dataou_unique = np.unique(dataou)
    num_classes   = dataou_unique.shape[0]

    fi      = fun_gaussian_epnn(datain_tr,datain_te,sigma, mode_val = 'tr')
    p_class = np.empty((num_te, fi.shape[1], fi.shape[2], num_classes))
    for i0 in range(num_classes):
      ind_ou            = np.where(dataou_tr == i0)[0]
      ind_class         = np.where(dataou_tr.repeat(num_te, axis = 1).transpose().reshape(num_tr * num_te,1)[:,0] == i0)[0]
      p_class[:,:,:,i0] = np.mean(fi[ind_class,:,:].reshape(num_te, ind_ou.shape[0], fi.shape[1], fi.shape[2]), axis = 1)

    class_val     = p_class.argmax(axis = 3)

    dataou_te     = np.expand_dims(dataou_te,axis = 2)
    dataou_te     = dataou_te.repeat(fi.shape[1], axis = 1)
    dataou_te     = dataou_te.repeat(fi.shape[2], axis = 2)
    correct       = class_val == dataou_te

    accuracy       = correct.sum(axis = 0)/num_te*100
    accuracy_tr    = accuracy.max() 

    ind_radius     = accuracy.argmax(axis = 0)
    max_radius     = accuracy.max(axis = 0)

    ind_sigma      = np.where(max_radius == accuracy_tr)[0]
    ind_radius     = np.unique(ind_radius[ind_sigma])

    #print(ind_radius)
    #print(ind_sigma)

    if ind_radius.shape[0] !=1 and ind_sigma.shape[0] !=1:
      sigma_optimum  = sigma[:,ind_radius,:] 
      sigma_optimum  = sigma_optimum[:,:,ind_sigma]
    else:
      sigma_optimum  = sigma[:,ind_radius,ind_sigma] 

    if ind_radius.shape[0] == 1:
      sigma_optimum = np.expand_dims(sigma_optimum, axis = 1)
    if ind_sigma.shape[0] == 1:
      sigma_optimum = np.expand_dims(sigma_optimum, axis = 2)

    return accuracy_tr, sigma_optimum

  def fun_epnn_test(datain_tr, dataou_tr, datain_te, dataou_te, sigma_optimum):
    num_tr    = datain_tr.shape[0]
    num_te    = datain_te.shape[0]

    sigma         = sigma_optimum
    dataou        = np.concatenate((dataou_tr, dataou_te), axis = 0)
    dataou_unique = np.unique(dataou)
    num_classes   = dataou_unique.shape[0]

    #print(datain_te.shape)
    fi      = fun_gaussian_epnn(datain_tr,datain_te,sigma, mode_val = 'te')
    #print(fi.shape)
    p_class = np.empty((num_te, fi.shape[1], fi.shape[2], num_classes))
    for i0 in range(num_classes):
      ind_ou            = np.where(dataou_tr == i0)[0]
      ind_class         = np.where(dataou_tr.repeat(num_te, axis = 1).transpose().reshape(num_tr * num_te,1)[:,0] == i0)[0]
      p_class[:,:,:,i0] = np.mean(fi[ind_class,:,:].reshape(num_te, ind_ou.shape[0], fi.shape[1], fi.shape[2]), axis = 1)

    class_val     = p_class.argmax(axis = 3)

    dataou_te     = np.expand_dims(dataou_te,axis = 2)
    dataou_te     = dataou_te.repeat(fi.shape[1], axis = 1)
    dataou_te     = dataou_te.repeat(fi.shape[2], axis = 2)
    correct       = class_val == dataou_te

    accuracy_te   = correct.sum(axis = 0)/num_te*100

    return accuracy_te


  accuracy_tr, sigma_optimum = fun_epnn_train(datain_tr, dataou_tr, sigma_resolution)
  accuracy_te                = fun_epnn_test(datain_tr, dataou_tr, datain_te, dataou_te, sigma_optimum)

  return accuracy_te, accuracy_tr, sigma_optimum

In [ ]:
#@title 6.4.2. Classification example: training and testing

sigma_resolution  = 0.1
radius_resolution = 0.1
accuracy_te, accuracy_tr, sigma_optimum = fun_epnn(datain_tr, dataou_tr, datain_te, dataou_te, sigma_resolution, radius_resolution)


print("##########################################################")
print("##########################################################")
print(" ")
print("EPNN Training Accuracy {}%".format(accuracy_tr))
print("EPNN Testing Accuracy  {}%".format(accuracy_te))
print(" ")
print("##########################################################")
print("##########################################################")
